# 内容说明
之前没进行量化测试，只是计算了测试集上的损失值观察下降趋势  
为了量化最终效果，计算NDCG指标，本文件采取了不同于前面的数据集划分策略以及评分值的利用方式  
1. 按照4:1根据评分文件划分训练集和测试集
2. 由于BPR针对隐式反馈，无法利用评分数值，因此将ml-100k中的评分为4或者5的当作positive，即为1，评分小于4或者未评分的当作negative，即为0  
3. 批训练集的构造：与前面相同，随机采样[u, i, j]三元组

In [31]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F

import pandas as pd
import random
import numpy as np
from collections import defaultdict

In [41]:
class BPR():
    
    def __init__(self, R_train, R_test, lr=0.005, weight_decay=0.05, embed_size=10, batch_size=500):
        
        # 常用参数
        self.n_P = R_train.size()[0]
        self.n_Q = R_train.size()[1]
        self.lr = lr  # 优化器参数
        self.weight_decay = weight_decay  # 优化器参数
        self.embed_size = embed_size
        self.batch_size = batch_size
        
        # 评分矩阵
        self.R_train = R_train
        self.R_test = R_test
        
        # 参数矩阵
        self.P = nn.Parameter(torch.empty(self.n_P, self.embed_size))
        self.Q = nn.Parameter(torch.empty(self.n_Q, self.embed_size))
        nn.init.xavier_normal_(self.P.data)
        nn.init.xavier_normal_(self.Q.data)
        
    
    # 生成批训练集，批大小为batch_size
    def generate_train_batch(self, rated_dict):
        
        # 记录一批中<u,i,j>的编号
        u_batch = []
        i_batch = []
        j_batch = []
    
        # 生成batch_size大小的训练集
        for b in range(self.batch_size):
            
            # 随机选择u，即user_id
            u = random.sample(rated_dict.keys(), 1)[0]
            u_batch.append(u)
            
            # 随机选择i，即该user已评分过的一个item_id
            i = random.sample(rated_dict[u], 1)[0]
            i_batch.append(i)
            
            # 随机选择j，即该user未评分过的一个item_id
            j = random.randint(1, n_Q-1)  # 随机生成item_id
            while j in rated_dict[u]:
                j = random.randint(1, n_Q-1)
            j_batch.append(j)
        
        # 以矩阵方式返回批训练集
        return np.asarray(u_batch), np.asarray(i_batch), np.asarray(j_batch)
    
    
    # 给定u, i, j编号矩阵，计算总误差值
    def compute_loss(self, u_array, i_array, j_array):
        pu = self.P[u_array, :]
        qi = self.Q[i_array, :]
        qj = self.Q[j_array, :]
        xui = torch.mul(pu, qi).sum(dim=1)
        xuj = torch.mul(pu, qj).sum(dim=1)
        xuij = xui - xuj
        log = F.logsigmoid(xuij).sum()
        loss = -log
        return loss
    
    
    # 训练
    def train(self, epochs, samples, rated_dict):
        optimizer = optim.Adam([self.P, self.Q], lr=self.lr, weight_decay=self.weight_decay)
        # 多次迭代
        print("\nstart training......")
        for k in range(epochs):
            sum_loss = 0
            # 每次迭代都有多次采样
            for n in range(1, samples):
                # 生成批训练集，计算损失值
                u_batch, i_batch, j_batch = self.generate_train_batch(rated_dict)
                loss = self.compute_loss(u_batch, i_batch, j_batch)
                sum_loss += loss
                # 优化参数
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()
            avg_loss = sum_loss/(self.batch_size*n)
            NDCG = self.test_NDCG(10)
            print('epoch %d：avg_loss = %.4f; test_NDCG = %.4f' % (k+1, avg_loss, NDCG))
    
    
    
    # 计算ndcg
    def test_NDCG(self, K):
        R_pred = torch.matmul(self.P, self.Q.t())
        R_pred -= 100*self.R_train  # 去除用于训练集的评分数据，减去100则排序后处于最后，不影响排序与计算
        
        sort_results1, indices1 = torch.sort(self.R_test, descending=True)
        sort_results2, indices2 = torch.sort(R_pred, descending=True)
        
        # 计算DCG，使用真实评分与预测的排序
        # 计算IDCG，使用真实评分与真实排序
        ndcg = 0
        n = self.n_P - 1
        for u in range(1, self.n_P):
            dcg = 0
            idcg = 0
            for idx in range(K):
                a = torch.tensor([idx+2])
                i = indices2[u][idx]
                dcg += (2**self.R_test[u][i]-1)/(torch.log2(a))
                idcg += (2**sort_results1[u][idx]-1)/(torch.log2(a))
            # 有的用户在测试数据中没评过分，idcg=0不可以做分母
            if idcg == 0:
                n -= 1
            else:
                ndcg += dcg / idcg  # 各用户的ndcg求和
        ndcg /= n  # 计算每个用户的平均ndcg
        
        return ndcg

In [42]:
# 定义函数，加载数据

def load_data(datapath):
    # 读取文件
    inter = pd.read_csv(data_path)
    df = pd.DataFrame(inter)
    
    # user, item数目+1
    n_P = df['user_id:token'].max() + 1
    n_Q = df['item_id:token'].max() + 1
    
    # 随机打乱，划分训练集与测试集
    df = df.sample(frac=1).reset_index(drop=True)
    n_train = int(0.8 * df.shape[0])
    df_train = df[:n_train]
    df_test = df[n_train:df.shape[0]]
    
    # 构造训练集与测试集的评分矩阵，>=4分则记1，否则都当作0
    # 构造训练集的已评分字典，便于bpr训练时分批采样
    R_train = torch.zeros(n_P, n_Q)
    rated_dict = defaultdict(set)
    R_test = torch.zeros(n_P, n_Q)
    for index, row in df_train.iterrows():
        if row['rating:float'] == 1:
            R_train[row['user_id:token']][row['item_id:token']] = 1
            u = row['user_id:token']
            i = row['item_id:token']
            rated_dict[u].add(i)
    for index, row in df_test.iterrows():
        if row['rating:float'] == 1:
            R_test[row['user_id:token']][row['item_id:token']] = 1
            
    # 返回
    return n_P, n_Q, R_train, R_test, rated_dict

## 2. 运行与测试

In [48]:
# 隐式数据集

# 读取文件，划分数据集
data_path = '../dataset/ml-100k/inter_implicit.csv'
n_P, n_Q, R_train, R_test, rated_dict = load_data(data_path)

# 构造模型并训练
bpr = BPR(R_train, R_test, lr=0.001, weight_decay=0.05, embed_size=48, batch_size=500)
bpr.train(10, 1000, rated_dict)


start training......
epoch 1：avg_loss = 0.4330; test_NDCG = 0.2145
epoch 2：avg_loss = 0.2520; test_NDCG = 0.2306
epoch 3：avg_loss = 0.2382; test_NDCG = 0.2412
epoch 4：avg_loss = 0.2324; test_NDCG = 0.2478
epoch 5：avg_loss = 0.2294; test_NDCG = 0.2535
epoch 6：avg_loss = 0.2278; test_NDCG = 0.2543
epoch 7：avg_loss = 0.2272; test_NDCG = 0.2543
epoch 8：avg_loss = 0.2259; test_NDCG = 0.2548
epoch 9：avg_loss = 0.2257; test_NDCG = 0.2558
epoch 10：avg_loss = 0.2257; test_NDCG = 0.2604
